### Rule-based chat bot



In [1]:
import nltk
import numpy as np
import random
import string

import bs4 as bs
import urllib.request
import re

In [2]:
raw_html = urllib.request.urlopen('https://en.wikipedia.org/wiki/Manchester_United_F.C.')
raw_html = raw_html.read()

In [4]:
article_html = bs.BeautifulSoup(raw_html, 'html')

In [5]:
article_para = article_html.find_all('p')


In [6]:
article_text = ''
for para in article_para:
    article_text += para.text

In [7]:
article_text = article_text.lower()

In [15]:
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

article_sentences = nltk.sent_tokenize(article_text)
article_words = nltk.word_tokenize(article_text)

In [16]:
wnlemmatizer = nltk.stem.WordNetLemmatizer()


def perform_lemmatization(tokens):
    return [wnlemmatizer.lemmatize(token) for token in tokens]

In [17]:
punctuation_removal = dict((ord(punct), None) for punct in string.punctuation)

In [18]:
def get_processed_text(document):
    return perform_lemmatization(nltk.word_tokenize(document.lower().translate(punctuation_removal)))

In [19]:
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup")
greeting_responses = ["hey", "hey hows you?", "*nods*", "hello, how you doing", "hello", "Welcome, I am good and you"]


def generate_greeting_response(greeting):
    for token in greeting.split():
        if token.lower() in greeting_inputs:
            return random.choice(greeting_responses)

In [20]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

following method generates a response to user queries. takes user input, finds cosine similarity of user input and compare with sentences in corpus

In [26]:
def generate_response(user_input):
    response = ''
    article_sentences.append(user_input)

    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words='english')
    all_word_vectorizer = word_vectorizer.fit_transform(article_sentences)

    # finds cosine similarity between the last item and word vector for all the sentences in corpus.
    similar_vector_values = cosine_similarity(all_word_vectorizer[-1], all_word_vectorizer)

    # sort list containing all cosine similarities of vectors, the second last item in the list will have the highest cosine(after sorting) with user input. the last item is the user input itself, so we dont select that
    similar_sentence_number = similar_vector_values.argsort()[0][-2]

    # flatten the retrieved cosine similarity and check if the similarity is equal to zero or not.
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = response + 'cant understand shit ya cunt'
        return response
    else:
        response = response + article_sentences[similar_sentence_number]
        return response

In [27]:

cont_dialogue = True
print('Hello Mfers. Ask me:')
while cont_dialogue:
    human_text = input()
    human_text = human_text.lower()
    if human_text != 'bye':
        if human_text == 'thanks' or human_text == 'thank you very much' or human_text == 'thank you':
            cont_dialogue = False
            print('Adios')
        else:
            if generate_greeting_response(human_text) is not None:
                print('Bot:' + generate_greeting_response(human_text))
            else:
                print("TennisRobo: ", end="")
                print(generate_response(human_text))
                article_sentences.remove(human_text)
    else:
        cont_dialogue = False
        print('Adiois')

Hello Mfers. Ask me:
TennisRobo: 

/home/lenovo/PycharmProjects/stackabuse/venv/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


cant understand shit ya cunt
TennisRobo: the rivalry with arsenal arises from the numerous times the two teams, as well as managers alex ferguson and arsène wenger, have battled for the premier league title.
TennisRobo: wayne rooney scored his 250th goal for united, a stoppage time equaliser in a league game against stoke city in january, surpassing sir bobby charlton as the club's all-time top scorer.
TennisRobo: the rivalry with arsenal arises from the numerous times the two teams, as well as managers alex ferguson and arsène wenger, have battled for the premier league title.
TennisRobo: wayne rooney scored his 250th goal for united, a stoppage time equaliser in a league game against stoke city in january, surpassing sir bobby charlton as the club's all-time top scorer.
TennisRobo: van gaal was ultimately sacked just two days after the cup final victory, with united having finished 5th in the league.
Adios
